<a href="https://colab.research.google.com/github/ppunkz/Portfoliobuilding/blob/master/Automating_Portfolio_Optimization_and_Allocation_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing all required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as web
from matplotlib.ticker import FuncFormatter

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [ ]:
pip install PyPortfolioOpt

     |████████████████████████████████| 51kB 1.5MB/s 


In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.cla import CLA
from pypfopt import plotting
from matplotlib.ticker import FuncFormatter

In [ ]:
tickers = ['AAV.BK','PTT.BK','AOT.BK','SCC.BK','BBL.BK','MINT.BK']
thelen = len(tickers)
price_data = []
for ticker in range(thelen):
  prices = web.DataReader(tickers[ticker], start='2015-01-01', end = '2020-06-06', data_source='yahoo')
  price_data.append(prices.assign(ticker=ticker)[['Adj Close']])
df_stocks = pd.concat(price_data, axis=1)
df_stocks.columns=tickers
df_stocks.head()

,AAV.BK,PTT.BK,AOT.BK,SCC.BK,BBL.BK,MINT.BK
Date,,,,,,
2015-01-05,3.756939,9.772616,17.388660,364.221069,152.504150,26.889683
2015-01-06,3.966629,10.020810,17.324968,357.745972,152.102829,25.831093
2015-01-07,4.001577,10.362075,18.025608,357.745972,153.708145,26.254513
2015-01-08,4.036525,10.548220,18.407776,362.602264,154.109482,27.313194
2015-01-09,4.123896,10.486171,18.535168,367.458527,152.504150,27.524950


In [ ]:
#Checking if any NaN values in the data
nullin_df = pd.DataFrame(df_stocks,columns=tickers)
print(nullin_df.isnull().sum())

AAV.BK     0
PTT.BK     0
AOT.BK     0
SCC.BK     0
BBL.BK     0
MINT.BK    0
dtype: int64


In [ ]:
#Annualized Return
mu = expected_returns.mean_historical_return(df_stocks)
#Sample Variance of Portfolio
Sigma = risk_models.sample_cov(df_stocks)

In [ ]:
#Max Sharpe Ratio - Tangent to the EF
ef = EfficientFrontier(mu, Sigma,  weight_bounds=(-1,1))
w = ef.max_sharpe()
print(ef.clean_weights())

OrderedDict([('AAV.BK', -0.15889), ('PTT.BK', 0.81116), ('AOT.BK', 1.0), ('SCC.BK', -0.22114), ('BBL.BK', -0.22777), ('MINT.BK', -0.20336)])


In [ ]:
from pypfopt import discrete_allocation, get_latest_prices

latest_prices = discrete_allocation.get_latest_prices(df_stocks)
# Allocate Portfolio Value in $ as required to show number of shares/stocks to buy, also bounds for shorting will affect allocation
#Min Volatility Portfolio Allocation $10000
allocation_minv, rem_minv = discrete_allocation.DiscreteAllocation(w, latest_prices, total_portfolio_value=10000).lp_portfolio()
print('Number of shares each stock you will buy and short to get minimum volatility in the portfolio')
print(allocation_minv)
print("Leftover Fund value in$ after building minimum volatility portfolio is ${:.2f}".format(rem_minv))

Number of shares each stock you will buy and short to get minimum volatility in the portfolio
{'PTT.BK': 114.0, 'AOT.BK': 82.0, 'AAV.BK': -252.0, 'SCC.BK': -2.0, 'BBL.BK': -7.0, 'MINT.BK': -35.0}
Leftover Fund value in$ after building minimum volatility portfolio is $29.11


In [ ]:
ef.portfolio_performance(verbose=True)

Expected annual return: 56.8%
Annual volatility: 41.6%
Sharpe Ratio: 1.32


(0.5676943607601772, 0.4164562514114982, 1.3151306023234677)